In [1]:
# Import Packages
from port_ctrl1 import Rigol_DCPort
from CAENDesktopHighVoltagePowerSupply1 import CAENDesktopHighVoltagePowerSupply, OneCAENChannel
import pyvisa as visa
import time
import numpy as np
import serial
import serial.tools.list_ports 
from matplotlib import pyplot as plt
from DC_Scan_class import *
from random import randint

import sys
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
from PyQt5.QtGui import QIcon
from PyQt5.QtCore import Qt, QTimer
from PyQt5.QtWidgets import *
from PyQt5 import QtCore, QtGui, QtWidgets
import pyqtgraph as pg

C:\Users\15421\anaconda3\lib\site-packages\pyqtgraph\colors\palette.py:1: RuntimeWarning: PyQtGraph supports Qt version >= 5.15, but 5.9.7 detected.
  from ..Qt import QtGui


In [2]:
class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(2500, 1500)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        
        # Connecting CAEN device, Button
        self.sendbutton = QtWidgets.QPushButton(self.centralwidget)
        self.sendbutton.setGeometry(QtCore.QRect(230, 280, 200, 50)) # upper left xy wh
        font = QtGui.QFont()
        font.setPointSize(12)
        self.sendbutton.setFont(font)
        self.sendbutton.setObjectName("sendbutton")
        
        # Configuration devices, Button
        self.confbutton = QtWidgets.QPushButton(self.centralwidget)
        self.confbutton.setGeometry(QtCore.QRect(125, 370, 150, 50)) # upper left xy wh
        font = QtGui.QFont()
        font.setPointSize(12)
        self.confbutton.setFont(font)
        self.confbutton.setObjectName("confbutton")
        
        # Cancel Configuration, Button
        self.cancelbutton = QtWidgets.QPushButton(self.centralwidget)
        self.cancelbutton.setGeometry(QtCore.QRect(285, 370, 150, 50)) # upper left xy wh
        font = QtGui.QFont()
        font.setPointSize(12)
        self.cancelbutton.setFont(font)
        self.cancelbutton.setObjectName("cancelbutton")
        self.cancelbutton.setEnabled(False)
        
        # Port Selection, Selection List
        self.sendcom = QtWidgets.QComboBox(self.centralwidget)
        self.sendcom.setGeometry(QtCore.QRect(280, 100, 150, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.sendcom.setFont(font)
        self.sendcom.setLayoutDirection(QtCore.Qt.LeftToRight)
        self.sendcom.setObjectName("sendcom")
        self.sendcom.addItem("")
        self.sendcom.setItemText(0, "")
        
        # Baud Rate Selection, Selection List
        self.sendbot = QtWidgets.QComboBox(self.centralwidget)
        self.sendbot.setGeometry(QtCore.QRect(280, 190, 150, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.sendbot.setFont(font)
        self.sendbot.setObjectName("sendbot")
        self.sendbot.addItem("")
        self.sendbot.addItem("")
        
        # Port, Label
        self.label = QtWidgets.QLabel(self.centralwidget)
        self.label.setGeometry(QtCore.QRect(125, 100, 70, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.label.setFont(font)
        self.label.setAlignment(QtCore.Qt.AlignCenter)
        self.label.setObjectName("label")
        
        # Baud Rate, Label
        self.label_2 = QtWidgets.QLabel(self.centralwidget)
        self.label_2.setGeometry(QtCore.QRect(60, 190, 200, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.label_2.setFont(font)
        self.label_2.setAlignment(QtCore.Qt.AlignCenter)
        self.label_2.setObjectName("label_2")
        
        # AQ Graph window
        #self.plotwindow = QtWidgets.QGroupBox(self.centralwidget)
        self.plotwindow = pg.PlotWidget()
        self.plotwindow.setGeometry(QtCore.QRect(60, 550, 1670, 900))
        self.plotwindow.setFont(font)
        self.plotwindow.setObjectName("plotwindow")
        
        # AQ Graph, Label
        self.label_3 = QtWidgets.QLabel(self.centralwidget)
        self.label_3.setGeometry(QtCore.QRect(60, 470, 160, 90))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.label_3.setFont(font)
        self.label_3.setAlignment(QtCore.Qt.AlignCenter)
        self.label_3.setObjectName("label_3")
        
        # AQ Graph Rectangle
        self.label_4 = QtWidgets.QLabel(self.centralwidget)
        self.label_4.setGeometry(QtCore.QRect(60, 550, 1670, 900))
        self.label_4.setFrameShape(QtWidgets.QFrame.Box)
        self.label_4.setFrameShadow(QtWidgets.QFrame.Plain)
        font = QtGui.QFont()
        self.label_4.setStyleSheet("background-color: White")
        font.setPointSize(12)
        self.label_4.setFont(font)
        self.label_4.setAlignment(QtCore.Qt.AlignCenter)
        self.label_4.setObjectName("label_4")
        self.label_4.lower()
        
        # Message Display window
        self.messagewindow = QtWidgets.QTextEdit(self.centralwidget)
        self.messagewindow.setGeometry(QtCore.QRect(1770, 550, 600, 900))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.messagewindow.setFont(font)
        self.messagewindow.setReadOnly(True)
        self.messagewindow.setObjectName("messagewindow")
        
        # Messages, Label
        self.messagelabel = QtWidgets.QLabel(self.centralwidget)
        self.messagelabel.setGeometry(QtCore.QRect(1770, 470, 160, 90))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.messagelabel.setFont(font)
        self.messagelabel.setAlignment(QtCore.Qt.AlignCenter)
        self.messagelabel.setObjectName("messagelabel")
        
        # Input 1:  RF starting amplitude, text input window
        self.Aminputdata = QtWidgets.QLineEdit(self.centralwidget)
        self.Aminputdata.setGeometry(QtCore.QRect(970, 100, 200, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.Aminputdata.setPlaceholderText("0 - 6 V")
        self.Aminputdata.setFont(font)
        self.Aminputdata.setObjectName("Aminputdata")
        self.Aminputdata.setReadOnly(True)
        reg1 = QtCore.QRegExp('[0-5]?\.\d{4}|6?')
        validator1 = QtGui.QRegExpValidator()
        validator1.setRegExp(reg1)
        self.Aminputdata.setValidator(validator1)
        
        # Input 1 Label
        self.Aminputlabel = QtWidgets.QLabel(self.centralwidget)
        self.Aminputlabel.setGeometry(QtCore.QRect(590, 100, 350, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.Aminputlabel.setFont(font)
        self.Aminputlabel.setAlignment(QtCore.Qt.AlignCenter)
        self.Aminputlabel.setObjectName("Aminputlabel")
        
        # Input 2: RF ending amplitude, text input window
        self.AMinputdata = QtWidgets.QLineEdit(self.centralwidget)
        self.AMinputdata.setGeometry(QtCore.QRect(970, 190, 200, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.AMinputdata.setPlaceholderText("0 - 6 V")
        self.AMinputdata.setFont(font)
        self.AMinputdata.setObjectName("AMinputdata")
        self.AMinputdata.setReadOnly(True)
        self.AMinputdata.setValidator(validator1)
        
        # Input 2 Label
        self.AMinputlabel = QtWidgets.QLabel(self.centralwidget)
        self.AMinputlabel.setGeometry(QtCore.QRect(590, 190, 350, 50)) 
        font = QtGui.QFont()
        font.setPointSize(12)
        self.AMinputlabel.setFont(font)
        self.AMinputlabel.setAlignment(QtCore.Qt.AlignCenter)
        self.AMinputlabel.setObjectName("AMinputlabel")
        
        # Input 3: DC lowest Volt, text input window
        self.Qminputdata = QtWidgets.QLineEdit(self.centralwidget)
        self.Qminputdata.setGeometry(QtCore.QRect(1570, 100, 200, 50)) 
        font = QtGui.QFont()
        font.setPointSize(12)
        self.Qminputdata.setPlaceholderText("0 - 3000 V")
        self.Qminputdata.setFont(font)
        self.Qminputdata.setObjectName("Qminputdata")
        self.Qminputdata.setReadOnly(True)
        reg2 = QtCore.QRegExp('\d?(\.\d{2})|[1-9]\d(\.\d{2})|[1-9]\d{2}(\.\d{2})|[1-2]\d{3}(\.\d{2})|3000?')
        validator2 = QtGui.QRegExpValidator()
        validator2.setRegExp(reg2)
        self.Qminputdata.setValidator(validator2)
        
        # Input 3 Label
        self.Qminputlabel = QtWidgets.QLabel(self.centralwidget)
        self.Qminputlabel.setGeometry(QtCore.QRect(1180, 100, 370, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.Qminputlabel.setFont(font)
        self.Qminputlabel.setAlignment(QtCore.Qt.AlignCenter)
        self.Qminputlabel.setObjectName("Qminputlabel")
        
        # Input 4: DC greatest Volt, text input window
        self.QMinputdata = QtWidgets.QLineEdit(self.centralwidget)
        self.QMinputdata.setGeometry(QtCore.QRect(1570, 190, 200, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.QMinputdata.setPlaceholderText("0 - 3000 V")
        self.QMinputdata.setFont(font)
        self.QMinputdata.setObjectName("QMinputdata")
        self.QMinputdata.setReadOnly(True)
        self.QMinputdata.setValidator(validator2)
        
        # Input 4 Label
        self.QMinputlabel = QtWidgets.QLabel(self.centralwidget)
        self.QMinputlabel.setGeometry(QtCore.QRect(1180, 200, 370, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.QMinputlabel.setFont(font)
        self.QMinputlabel.setAlignment(QtCore.Qt.AlignCenter)
        self.QMinputlabel.setObjectName("QMinputlabel")
        
        # Input 5: Hstep
        self.Hstep = QtWidgets.QLineEdit(self.centralwidget)
        self.Hstep.setGeometry(QtCore.QRect(1570, 370, 200, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.Hstep.setPlaceholderText("sep by ','")
        self.Hstep.setFont(font)
        self.Hstep.setObjectName("Hstep")
        self.Hstep.setReadOnly(True)
        self.Hstep.setValidator(validator2)
        reg4 = QtCore.QRegExp("^(((0(\.\d{1,2})?)|([1-9](\.\d{0,2})?)|([1-9]\d(\.\d{0,2})?)|([1-9]\d\d(\.\d{0,2})?)|([1-2]\d\d\d(\.\d{0,2})?)|(3000)))(,\s*((0(\.\d{1,2})?)|([1-9](\.\d{0,2})?)|([1-9]\d(\.\d{0,2})?)|([1-9]\d\d(\.\d{0,2})?)|([1-2]\d\d\d(\.\d{0,2})?)|(3000)))*$")
        validator4 = QtGui.QRegExpValidator()
        validator4.setRegExp(reg4)
        self.Hstep.setValidator(validator4)
        
        # Input 5 Hstep Label
        self.Hsteplabel = QtWidgets.QLabel(self.centralwidget)
        self.Hsteplabel.setGeometry(QtCore.QRect(1210, 370, 300, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.Hsteplabel.setFont(font)
        self.Hsteplabel.setAlignment(QtCore.Qt.AlignCenter)
        self.Hsteplabel.setObjectName("Hsteplabel")
        
        # Input 6: Astep
        self.Astep = QtWidgets.QLineEdit(self.centralwidget)
        self.Astep.setGeometry(QtCore.QRect(970, 370, 200, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.Astep.setPlaceholderText("sep by ','")
        self.Astep.setFont(font)
        self.Astep.setObjectName("Astep")
        self.Astep.setReadOnly(True)
        reg3 = QtCore.QRegExp("^(((0(\.\d{1,4})?)|([1-5](\.\d{0,4})?))|(6))(,\s*((0(\.\d{1,4})?)|([1-5](\.\d{0,4})?))|(6))*$")
        validator3 = QtGui.QRegExpValidator()
        validator3.setRegExp(reg3)
        self.Astep.setValidator(validator3)
        
        # Input 6 Astep Label
        self.Asteplabel = QtWidgets.QLabel(self.centralwidget)
        self.Asteplabel.setGeometry(QtCore.QRect(620, 370, 305, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.Asteplabel.setFont(font)
        self.Asteplabel.setAlignment(QtCore.Qt.AlignCenter)
        self.Asteplabel.setObjectName("Asteplabel")
        
        # Input 5: Channel for RF
        self.AChannel = QtWidgets.QComboBox(self.centralwidget)
        self.AChannel.setGeometry(QtCore.QRect(2160, 100, 200, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.AChannel.setFont(font)
        self.AChannel.setObjectName("AChannel")
        self.AChannel.addItem("")
        self.AChannel.addItem("")
        self.AChannel.addItem("")
        self.AChannel.setEnabled(False)
        
        # Input 5 Label
        self.AChannellabel = QtWidgets.QLabel(self.centralwidget)
        self.AChannellabel.setGeometry(QtCore.QRect(1800, 100, 300, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.AChannellabel.setFont(font)
        self.AChannellabel.setAlignment(QtCore.Qt.AlignCenter)
        self.AChannellabel.setObjectName("AChannellabel")
        
        # Input 6: Channel1 for DC
        self.QChannel = QtWidgets.QComboBox(self.centralwidget)
        self.QChannel.setGeometry(QtCore.QRect(2160, 190, 200, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.QChannel.setFont(font)
        self.QChannel.setObjectName("QChannel")
        self.QChannel.addItem("")
        self.QChannel.addItem("")
        self.QChannel.addItem("")
        self.QChannel.addItem("")
        self.QChannel.setEnabled(False)
        
        # Input 6 Label
        self.QChannellabel = QtWidgets.QLabel(self.centralwidget)
        self.QChannellabel.setGeometry(QtCore.QRect(1800, 190, 305, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.QChannellabel.setFont(font)
        self.QChannellabel.setAlignment(QtCore.Qt.AlignCenter)
        self.QChannellabel.setObjectName("QChannellabel")
        
        # Input 6: Channel2 for DC
        self.QChannel1 = QtWidgets.QComboBox(self.centralwidget)
        self.QChannel1.setGeometry(QtCore.QRect(2160, 280, 200, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.QChannel1.setFont(font)
        self.QChannel1.setObjectName("QChannel")
        self.QChannel1.addItem("")
        self.QChannel1.addItem("")
        self.QChannel1.addItem("")
        self.QChannel1.addItem("")
        self.QChannel1.setEnabled(False)
        
        # Input 6 Label Channel 2
        self.QChannel1label = QtWidgets.QLabel(self.centralwidget)
        self.QChannel1label.setGeometry(QtCore.QRect(1800, 280, 305, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.QChannel1label.setFont(font)
        self.QChannel1label.setAlignment(QtCore.Qt.AlignCenter)
        self.QChannel1label.setObjectName("QChannel1label")
        
        # Input 7: Number of Sampling Points
        self.Sample = QtWidgets.QLineEdit(self.centralwidget)
        self.Sample.setGeometry(QtCore.QRect(970, 280, 200, 50)) #1570, 300, 190, 50
        font = QtGui.QFont()
        font.setPointSize(12)
        self.Sample.setFont(font)
        self.Sample.setObjectName("Sample")
        self.Sample.setReadOnly(True)
        reg5 = QtCore.QRegExp("^[1-9]\d*$")
        validator5 = QtGui.QRegExpValidator()
        validator5.setRegExp(reg5)
        self.Sample.setValidator(validator5)
        
        # Input 7 Label
        self.Samplelabel = QtWidgets.QLabel(self.centralwidget)
        self.Samplelabel.setGeometry(QtCore.QRect(620, 280, 300, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.Samplelabel.setFont(font)
        self.Samplelabel.setAlignment(QtCore.Qt.AlignCenter)
        self.Samplelabel.setObjectName("Samplelabel")
        
        # Input 8: Duration
        self.Duration = QtWidgets.QLineEdit(self.centralwidget)
        self.Duration.setGeometry(QtCore.QRect(1570, 280, 200, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.Duration.setFont(font)
        self.Duration.setObjectName("Sample")
        self.Duration.setReadOnly(True)
        reg6 = QtCore.QRegExp("^((0\.\d{1,3})|([1-9]\d*(\.\d{1,3})?))(,\s*((0\.\d{1,3})|([1-9]\d*(\.\d{1,3})?)))*$")
        validator6 = QtGui.QRegExpValidator(reg6)
        validator6.setRegExp(reg6)
        self.Duration.setValidator(validator6)
        self.QMinputdata.setPlaceholderText("Up to 3 deci")
        
        # Input 8 Label
        self.Durationlabel = QtWidgets.QLabel(self.centralwidget)
        self.Durationlabel.setGeometry(QtCore.QRect(1210, 280, 300, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.Durationlabel.setFont(font)
        self.Durationlabel.setAlignment(QtCore.Qt.AlignCenter)
        self.Durationlabel.setObjectName("Durationlabel")
        
        # Rectangle for Communication Ports
        self.Com_portal = QtWidgets.QLabel(self.centralwidget)
        self.Com_portal.setGeometry(QtCore.QRect(60, 90, 400, 250))
        self.Com_portal.setFrameShape(QtWidgets.QFrame.Box)
        self.Com_portal.setFrameShadow(QtWidgets.QFrame.Plain)
        self.Com_portal.setAlignment(QtCore.Qt.AlignCenter)
        self.Com_portal.lower()
        
        # Label for Communication Ports
        self.Com_portallabel = QtWidgets.QLabel(self.centralwidget)
        self.Com_portallabel.setGeometry(QtCore.QRect(60, 50, 300, 50))
        self.Com_portallabel.setAlignment(QtCore.Qt.AlignCenter)
        
        # Rectangle for Parameter Settings
        self.Par_set = QtWidgets.QLabel(self.centralwidget)
        self.Par_set.setGeometry(QtCore.QRect(570, 90, 1800, 350))
        self.Par_set.setFrameShape(QtWidgets.QFrame.Box)
        self.Par_set.setFrameShadow(QtWidgets.QFrame.Plain)
        self.Par_set.setAlignment(QtCore.Qt.AlignCenter)
        self.Par_set.lower()
        
        # Label for Parameter Settings
        self.Par_setlabel = QtWidgets.QLabel(self.centralwidget)
        self.Par_setlabel.setGeometry(QtCore.QRect(570, 50, 350, 50))
        self.Par_setlabel.setAlignment(QtCore.Qt.AlignCenter)
        
        # Mode Selection
        self.custom = QtWidgets.QCheckBox(self.centralwidget)
        self.custom.setGeometry(QtCore.QRect(2100, 370, 220, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.custom.setFont(font)
        self.custom.setLayoutDirection(QtCore.Qt.LeftToRight)
        self.custom.setObjectName("custom")
        
        self.auto = QtWidgets.QCheckBox(self.centralwidget)
        self.auto.setGeometry(QtCore.QRect(1850, 370, 210, 50))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.auto.setFont(font)
        self.auto.setLayoutDirection(QtCore.Qt.LeftToRight)
        self.auto.setObjectName("auto")
        
        self.custom.stateChanged.connect(self.mode_select)
        self.auto.stateChanged.connect(self.mode_select)
        
        # Display all designs above in the window
        MainWindow.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(MainWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 775, 18))
        self.menubar.setObjectName("menubar")
        
        # Headline Menu
        MainWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)
        
        # Enable buttons and Plots in UI
        self.retranslateUi(MainWindow)
        # Connecting port with preset baudrate and port
        self.sendbutton.clicked.connect(MainWindow.open_com)
        self.sendcom.activated['QString'].connect(MainWindow.port_changed)
        self.sendbot.activated['QString'].connect(MainWindow.baud_changed)
        
        self.confbutton.clicked.connect(MainWindow.config)
        self.cancelbutton.clicked.connect(MainWindow.cancel)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)    

In [3]:
class Ui_MainWindow(Ui_MainWindow):    
    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "Amplitude Scan Control Panel"))
        self.sendbutton.setStatusTip(_translate("MainWindow", "Click to Connect"))
        self.sendbutton.setText(_translate("MainWindow", "Port Conn"))
        self.sendbot.setItemText(0, _translate("MainWindow", "9600"))
        self.sendbot.setItemText(1, _translate("MainWindow", "115200"))
        
        #self.AChannel.setItemText(0, _translate("MainWindow", ""))
        self.AChannel.setItemText(0, _translate("MainWindow", "CH1"))
        self.AChannel.setItemText(1, _translate("MainWindow", "CH2"))
        self.AChannel.setItemText(2, _translate("MainWindow", "CH3"))
        
        #self.QChannel.setItemText(0, _translate("MainWindow", ""))
        self.QChannel.setItemText(0, _translate("MainWindow", "CH0"))
        self.QChannel.setItemText(1, _translate("MainWindow", "CH1"))
        self.QChannel.setItemText(2, _translate("MainWindow", "CH2"))
        self.QChannel.setItemText(3, _translate("MainWindow", "CH3"))
        
        #self.QChannel1.setItemText(0, _translate("MainWindow", ""))
        self.QChannel1.setItemText(0, _translate("MainWindow", "CH4"))
        self.QChannel1.setItemText(1, _translate("MainWindow", "CH5"))
        self.QChannel1.setItemText(2, _translate("MainWindow", "CH6"))
        self.QChannel1.setItemText(3, _translate("MainWindow", "CH7"))
        
        self.label.setText(_translate("MainWindow", "Port"))
        self.label_2.setText(_translate("MainWindow", "Baud Rate"))
        self.label_3.setText(_translate("MainWindow", "A-Q Graph"))
        
        self.Aminputlabel.setText(_translate("MainWindow", "Start Amplitude (V):"))
        self.AMinputlabel.setText(_translate("MainWindow", "End Amplitude (V):"))
        self.Qminputlabel.setText(_translate("MainWindow", "Start DC Voltage (V):"))
        self.QMinputlabel.setText(_translate("MainWindow", "End DC Voltage (V):"))
        self.AChannellabel.setText(_translate("MainWindow", "Channel for RF:"))
        self.QChannellabel.setText(_translate("MainWindow", "Channel 1 for DC:"))
        self.QChannel1label.setText(_translate("MainWindow", "Channel 2 for DC:"))

        self.Asteplabel.setText(_translate("MainWindow", "RF Step (V):"))
        self.Hsteplabel.setText(_translate("MainWindow", "DC Step (V):"))
        
        self.Samplelabel.setText(_translate("MainWindow", "Number of Samples:"))
        self.Com_portallabel.setText(_translate("MainWindow", "Communication Settings"))
        self.Par_setlabel.setText(_translate("MainWindow", "Scanning Parameter Settings"))
        self.confbutton.setText(_translate("MainWindow", "Configure"))
        self.cancelbutton.setText(_translate("MainWindow", "Cancel"))
        self.messagelabel.setText(_translate("MainWindow", "Messages"))
        self.Durationlabel.setText(_translate("MainWindow", "Duration (s):"))
        
        self.custom.setText(_translate("MainWindow", "Custom Mode"))
        self.auto.setText(_translate("MainWindow", "Auto Mode"))

In [4]:
class Ui_MainWindow(Ui_MainWindow):
    def mode_select(self, state):
        # If either check box is checked, disable the other
        if state == Qt.Checked:
            if self.sender() == self.custom:
                self.auto.setChecked(False)
                self.AChannel.setEnabled(True)
                self.QChannel.setEnabled(True)
                self.QChannel1.setEnabled(True)
                self.Aminputdata.setReadOnly(False)
                self.AMinputdata.setReadOnly(False)
                self.Qminputdata.setReadOnly(False)
                self.QMinputdata.setReadOnly(False)
                self.Duration.setReadOnly(False)
                self.Astep.setReadOnly(False)
                self.Hstep.setReadOnly(False)
                self.Sample.setReadOnly(True)
            
            if self.sender() == self.auto:
                self.custom.setChecked(False)
                self.Aminputdata.setReadOnly(False)
                self.AMinputdata.setReadOnly(False)
                self.Qminputdata.setReadOnly(False)
                self.QMinputdata.setReadOnly(False)
                self.Sample.setReadOnly(False)
                self.Duration.setReadOnly(False)
                self.AChannel.setEnabled(True)
                self.QChannel.setEnabled(True)
                self.QChannel1.setEnabled(True)
                self.Astep.setReadOnly(True)
                self.Hstep.setReadOnly(True)
        else:
            self.AChannel.setEnabled(False)
            self.QChannel.setEnabled(False)
            self.QChannel1.setEnabled(False)
            self.Aminputdata.setReadOnly(True)
            self.AMinputdata.setReadOnly(True)
            self.Qminputdata.setReadOnly(True)
            self.QMinputdata.setReadOnly(True)
            self.Duration.setReadOnly(True)
            self.Astep.setReadOnly(True)
            self.Hstep.setReadOnly(True)
            self.Sample.setReadOnly(True)

In [5]:
def get_com_list():
    Com_List = []
    plist = list(serial.tools.list_ports.comports())
    if len(plist) > 0:
        for i in range(len(plist)):
            Com_List.append(list(plist[i])[0])
    return Com_List

In [6]:
#Logic
class MyWindow(QMainWindow,Ui_MainWindow):
    def __init__(self, parent=None):
        super(MyWindow, self).__init__(parent)
        self.setupUi(self)
        self.data0 = None
        self.data1 = []
        self.last_txt=None
        self.setWindowIcon(QIcon('serialscope.ico'))
        self.sendbutton.setToolTip('Click to Open the port')
        #self.curr_v = []
        #self.curr_u = []
        #self.curr_setv = []
        #self.curr_setu = []
        self.time = list(range(10))
        self.temperature = [randint(20, 40) for _ in range(10)]
        self.plotwindow.setBackground("w")
        self.plotwindow.setTitle("A vs Q", color="b", size="20pt")
        self.plotwindow.plotItem.setLabel("bottom",text="RF Amplitude (V)")
        self.plotwindow.plotItem.setLabel("left",text="DC Voltage (V)")
        self.plotwindow.addLegend()
        self.plotwindow.showGrid(x=True, y=True)
        self.line = self.plotwindow.plot(
            self.time,
            self.temperature,
            name="Setting Values"
        )
        self.timer = pg.QtCore.QTimer(self)
        self.timer.timeout.connect(self.updateData)
        self.timer.start()
        #self.curve1 = self.plotwindow.plot(x=self.curr_v, y=self.curr_u)
        #self.curve2 = self.plotwindow.plot(x=self.curr_setv, y=self.curr_setu)

    def show_dialog(self,str,title=""):
        # Create QDialog object
        dialog = QDialog()
        layout = QVBoxLayout(dialog)
        label_layout = QHBoxLayout()
        button_layout = QHBoxLayout()
        
        # Create QLabel and QPushButton in the dialog
        lb = QLabel(str, dialog)
        btn = QPushButton('ok', dialog)
        
        lb.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)
        btn.setSizePolicy(QSizePolicy.Fixed, QSizePolicy.Fixed)
        
        label_layout.addWidget(lb)
        button_layout.addWidget(btn)
        label_layout.setAlignment(Qt.AlignCenter)
        
        layout.addLayout(label_layout)
        layout.addLayout(button_layout)
        btn.clicked.connect(dialog.accept)
    
        # Set dialog title
        dialog.setWindowTitle(title)
        
        # Set the window modality and adjust the size to fit the contents
        dialog.setWindowModality(Qt.NonModal)
        dialog.adjustSize()
        
        # Show the dialog
        dialog.exec_()
    
    #######################
    def open_com(self):
        global ser, serialPort, baudRate, com_list
        com_list = get_com_list() # Get available Ports  
        DC.working_message="Checking Connection..."
        self.messagewindow.append(DC.working_message)
        if com_list != []:  
            if serialPort != None and self.sendcom.currentText() != "":# Port is not None and Connect button is clicked
                ser.port = serialPort
                ser.baudrate = baudRate
                if self.sendbutton.text() == 'Connect':
                    ser.open() # Cannot open an open port
                    #self.timer.start(timer_value) #Start Timer for message display
                    DC.working_message="Checking Connection Complete"
                    self.messagewindow.append(DC.working_message)
                    self.sendcom.setEnabled(False)
                    self.sendbot.setEnabled(False)
                    self.sendbutton.setText("Disconnect")
                    self.sendbutton.setToolTip('Click to Close Port')
                    print('Port Connect')
                else:
                    #self.timer.stop()
                    ser.close()  # Close the port
                    self.sendcom.setEnabled(True)
                    self.sendbot.setEnabled(True)
                    self.sendbutton.setText("Connect")
                    self.sendbutton.setToolTip('Click to Open Port')
                    print('Port Disconnect')
            elif serialPort != None and self.sendcom.currentText() == "":
                myWin.show_dialog(str='Please Choose a Port', title='Warning：No Port Chosen')
            else:  # Fixing
                myWin.port_combo.clear()
                for i in range(len(com_list)):  # Add available Ports to combo box
                    myWin.sendcom.addItem(com_list[i])
                serialPort = com_list[0]  # Setting the first available port by default
        else:  # Show warning dialog
            serialPort = None
            myWin.sendcom.clear()
            myWin.show_dialog(str='No Port Detected', title='Warning：Port Unfound')
            
    def cancel(self):
        #self.timer.stop()
        ser.close()
        self.sendcom.setEnabled(True)
        self.sendbot.setEnabled(True)
        self.sendbutton.setText("Connect")
        self.sendbutton.setToolTip('Click to Open Port')
        print('Port Disconnect')
        self.confbutton.setEnabled(True)
        self.cancelbutton.setEnabled(False)
        self.auto.setEnabled(True)
        self.custom.setEnabled(True)
        self.auto.setChecked(False)
        self.custom.setChecked(False)
        self.AChannel.setEnabled(False)
        self.QChannel.setEnabled(False)
        self.QChannel1.setEnabled(False)
        self.Aminputdata.setReadOnly(True)
        self.AMinputdata.setReadOnly(True)
        self.Qminputdata.setReadOnly(True)
        self.QMinputdata.setReadOnly(True)
        self.Duration.setReadOnly(True)
        self.Astep.setReadOnly(True)
        self.Hstep.setReadOnly(True)
        self.Sample.setReadOnly(True)
        #DC.cancel_configuration()
        DC.working_message="Current Program Canceled"
        self.messagewindow.append(DC.working_message)
        
    def config(self):
        global serialPort, DC
        if serialPort != None :
            if self.sendbutton.text() == 'Disconnect'and self.sendcom.currentText() != "":
                DC.working_message="Checking Mode Configuration..."
                self.messagewindow.append(DC.working_message)
                # Check if inputs are valid
                self.confbutton.setEnabled(False)
                self.cancelbutton.setEnabled(True)
                result = self.Check_scan()
                if result == False:
                    DC.error_message=("Error occurs, configure cancelled.")
                    self.messagewindow.append(DC.error_message)
                    self.cancel()
                else:
                    DC.working_message="Configuration Complete.\n" + "Start Scanning..."
                    self.messagewindow.append(DC.working_message)
                    self.auto.setEnabled(False)
                    self.custom.setEnabled(False)
                    self.AChannel.setEnabled(False)
                    self.QChannel.setEnabled(False)
                    self.QChannel1.setEnabled(False)
                    self.Aminputdata.setReadOnly(True)
                    self.AMinputdata.setReadOnly(True)
                    self.Qminputdata.setReadOnly(True)
                    self.QMinputdata.setReadOnly(True)
                    self.Duration.setReadOnly(True)
                    self.Astep.setReadOnly(True)
                    self.Hstep.setReadOnly(True)
                    self.Sample.setReadOnly(True)
                    if self.auto.isChecked():
                        try:
                            self.Auto_DC_Scan()
                            self.timer.stop()
                            DC.working_message="Scanning Complete."
                            self.messagewindow.append(DC.working_message)
                            self.confbutton.setEnabled(True)
                            self.cancelbutton.setEnabled(False)
                            ser.close()
                            self.sendcom.setEnabled(True)
                            self.sendbot.setEnabled(True)
                            self.sendbutton.setText("Connect")
                            self.sendbutton.setToolTip('Click to Open Port')
                            self.confbutton.setEnabled(True)
                            self.cancelbutton.setEnabled(False)
                            self.auto.setEnabled(True)
                            self.custom.setEnabled(True)
                            self.auto.setChecked(False)
                            self.custom.setChecked(False)
                            self.AChannel.setEnabled(False)
                            self.QChannel.setEnabled(False)
                            self.QChannel1.setEnabled(False)
                            self.Aminputdata.setReadOnly(True)
                            self.AMinputdata.setReadOnly(True)
                            self.Qminputdata.setReadOnly(True)
                            self.QMinputdata.setReadOnly(True)
                            self.Duration.setReadOnly(True)
                            self.Astep.setReadOnly(True)
                            self.Hstep.setReadOnly(True)
                            self.Sample.setReadOnly(True)
                        except:
                            try:
                                DC.error_message="Error Ocurrs, Scanning Canceled."
                                self.messagewindow.append(DC.error_message)
                                self.cancel()
                            except:
                                self.show_dialog(str='DC Devices Not Found',title="Warning: No DC Devices")
                    
            elif (self.sendbutton.text() == 'Connect' or self.sendbutton.text() == 'Port Conn')and self.sendcom.currentText() != "":
                self.show_dialog(str='Please Connect to the Port',title="Warning: No Connection")
            else:
                self.show_dialog(str='Please Choose a Port',title="Warning: No Port Selected")
        else:
            self.show_dialog(str='No Port Found',title="Warning: No Port Selected")
    
    def Check_scan(self):
        global DC
        if not self.auto.isChecked() and not self.custom.isChecked():
            self.show_dialog(str='Please Choose a Mode',title="Warning: No Mode Selected")
            DC.error_message="No Mode Selected"
            self.messagewindow.append(DC.error_message)
            return False
        else:
            DC.working_message="Mode Check Complete\n" + "Start RF and DC Check"
            self.messagewindow.append(DC.working_message)
        
        try:
            DC.Arange = [float(self.Aminputdata.text()), float(self.AMinputdata.text())]
            DC.Hrange = [float(self.Qminputdata.text()), float(self.QMinputdata.text())]
            DC.working_message="RF and DC Check Complete\n" + "Start Channel Check"
            self.messagewindow.append(DC.working_message)
        except:
            self.show_dialog(str='Amplitude or DC unspecified',title="Warning: Invalid Input")
            DC.error_message="RF or DC Unspecified"
            self.messagewindow.append(DC.error_message)
            return False
        
        try:
            int(self.AChannel.currentText()[-1])
            int(self.QChannel.currentText()[-1])
            int(self.QChannel1.currentText()[-1])
            DC.Achannel = self.AChannel.currentText()
            DC.Hchannel = self.QChannel.currentText()
            DC.Hchannel1 = self.QChannel1.currentText()
            DC.working_message="Channel Check Complete\n" + "Start Mode Parameter Check"
            self.messagewindow.append(DC.working_message)
        except:
            self.show_dialog(str='Channels unspecified',title="Warning: Invalid Input")
            DC.error_message="Channel Unspecified"
            self.messagewindow.append(DC.error_message)
            return False
        
        if self.auto.isChecked():
            DC.mode=0
            try:
                DC.sampling=int(self.Sample.text())
                DC.sampling_validity() # Setting curr_q, curr_a, sampling
                DC.working_message="Parameter Check Complete\n" + "Start Duration Check..."
                self.messagewindow.append(DC.working_message)
            except:    
                self.show_dialog(str='Number of Samples Unspecified',title="Warning: Invalid Input")
                return False
        elif self.custom.isChecked():
            DC.mode=1
            try:
                DC.Astep=float(self.Astep.text())
                DC.Hstep=float(self.Hstep.text())
                DC.step_validity()
                DC.working_message="Parameter Check Complete\n" + "Start Duration Check..."
                self.messagewindow.append(DC.working_message)
            except:
                try:
                    DC.Astep=map(float,self.Astep.text().split(','))
                    DC.Hstep=map(float,self.Hstep.text().split(','))
                    
                    if len(DC.Astep) != len(DC.Hstep):
                        self.show_dialog(str='Number of Steps Mismatch',title="Warning: Invalid Input")
                        DC.error_message="Step Mismatch"
                        self.messagewindow.append(DC.error_message)
                        return False
                except:
                    self.show_dialog(str='Error in Steps Input',title="Warning: Invalid Input")
                    DC.error_message="Invalid Step Input"
                    self.messagewindow.append(DC.error_message)
                    return False
            
        try:
            DC.intv = int(self.Duration.text())
            DC.duration_check()
            DC.working_message="Duration Check Complete"
            self.messagewindow.append(DC.working_message)
        except:
            try:
                if self.Duration.text()=="":
                    DC.intv=None
                    DC.duration_check()
                else:
                    Duration = self.Duration.text().split(',')
                    if Duration[-1] == "":
                        Duration.pop()
                    for i in range(len(Duration)):
                        Duration[i] = int(Duration[i])
                    DC.intv = Duration
                    DC.duration_check()
            except:
                self.show_dialog(str='Number of Duration Mismatch',title="Warning: Invalid Input")
                DC.error_message="Duration Mismatch"
                self.messagewindow.append(DC.error_message)
                return False
        return True
    
    def updateData(self):
        #self.curve1.setData(self.curr_v, self.curr_u)
        #self.curve2.setData(self.curr_setv, self.curr_setu)
        #time.sleep(0.1)
        #self.index+=1
        #self.curr_setv.append(DC.curr_a[self.index])
        #self.curr_setu.append(DC.curr_q[self.index])
        #self.line.setData(self.curr_setv, self.curr_setu)
        #if self.index==DC.sampling-1:
        #    self.timer.stop()
        self.time = self.time[1:]
        self.time.append(self.time[-1] + 1)
        self.temperature = self.temperature[1:]
        self.temperature.append(randint(20, 40))
        self.line.setData(self.time, self.temperature)
    
    def Auto_DC_Scan(self):
        global DC
        self.curr_v = [DC.curr_a[0]]
        self.curr_u = [DC.curr_q[0]]
        self.curr_setv=[DC.curr_a[0]]
        self.curr_setu=[DC.curr_q[0]]
        self.index=0
        #self.timer.start(1000)
        
        #F=MyFigure()
        #axes=F.fig.add_subplot(111)
        #t = np.arange(0.0,5.0,0.01)
        #s = np.cos(2*np.pi*t)
        #axes.plot(t,s)
        #QtWidgets.QGridLayout(self.plotwindow).addWidget(F)
        
        ## Initialization
        #DC.RF_set(curr_a[0])
        #DC.DC_set(curr_q[0],DC.Hchannel)
        #DC.DC_set(curr_q[0],DC.Hchannel1)
        #DC.DC_channel(state="ON", channel=DC.Hchannel)
        #DC.DC_channel(state="ON", channel=DC.Hchannel1)
        #time.sleep(30) # Waiting for Instrument Configuration
        #DC.RF_channel(state="ON")
        #
        #for i in range(DC.sampling):
        #    DC.RF.set_dc_fix_value(curr_a[i], 2, channel=DC.Achannel)
        #    DC.DC.query(CMD="SET", PAR="VSET", CH=DC.Hchannel[-1], VAL=DC.curr_q[i])
        #    DC.DC.query(CMD="SET", PAR="VSET", CH=DC.Hchannel1[-1], VAL=DC.curr_q[i])
        #    self.curr_setv.append(DC.curr_a[i])
        #    self.curr_setu.append(DC.curr_q[i])
        #    
        #   for _ in range(int(DC.interval[i])):
        #       time.sleep(1)
        #        try:
        #            Hmon = H.get_single_channel_parameter("VMON", Hchannel[-1])
        #            Hmon = float(Hmon)
        #        except:
        #            time.sleep(1)
        #            continue
        #        
        #        curr_u.append(Hmon)
        #        curr_v.append(A.read_dc_v(channel = Achannel))
        #        
                 
             
             
        #        axes.clear()
        #        axes.plot(curr_setv, curr_setu,'ro-',label="Set Values")
        #        axes.plot(curr_v, curr_u,'bo-',label="Monitor Value")
        #        axes.set_xlabel('RF Amplitude')
        #        axes.set_ylabel('DC Voltage')
        #        axes.set_title('Q vs A')
        #        time.sleep(1)
        #
        #DC.cancel_configuration()
        #time.sleep(30) # Waiting for entire set to reset
                
    #def Custom_Scan(self):
    #    global DC        
    
    def port_changed(self, text):
        global ser, serialPort
        serialPort = text
        print('Port：' + serialPort)
        ser.port = serialPort
        
    def baud_changed(self, text):
        global ser, baudRate
        baudRate = int(text)
        print(baudRate)
        ser.baudrate=baudRate

In [7]:
####################################################
#Gloabal Variables and Initialization
app = QApplication(sys.argv)
myWin = MyWindow()
myWin.show()
#myWin.Port_send()
baudRate = 9600  # baudrate
timer_value=1
#################################################
com_list=get_com_list() #obtain port list

ser=serial.Serial(timeout=1)
DC = DC_Scan(ser)

if com_list!=[]:
    for i in range(len(com_list)):
        myWin.sendcom.addItem(com_list[i])
    serialPort = com_list[0]
else:
    serialPort = None
    myWin.show_dialog(str='Please Open the Port')

sys.exit(app.exec_())

Port：COM3
Port Disconnect
Port Connect


SystemExit: 0

C:\Users\15421\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
